In [1]:
import pandas as pd
import numpy as np
import emoji
import re
import requests
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
emoji_dict = {
    "red_heart": "Just loved it",
    "pizza": "pizza",
    "soft_ice_cream": "ice cream",
    "OK_hand_light_skin_tone":"Okay",
    "OK_hand":"Okay",
    "OK_hand_dark_skin_tone":"Okay",
    "OK_button": "Okay",
    "OK_hand_medium_skin_tone": "Okay",
    "anger_symbol": "Very angry",
    "angry_face":"Anger and Dissatisfied",
    "angry_face_with_horns":"Anger and Dissatisfied",
    "anxious_face_with_sweat": "Anxious and Not happy",
    "astonished_face":"Astonishing",
    "clapping_hands":"Awesome! Very well done",
    "clapping_hands_dark_skin_tone":"Awesome! Very well done",
    "clapping_hands_light_skin_tone":"Awesome! Very well done",
    "clapping_hands_medium-dark_skin_tone":"Awesome! Very well done",
    "clapping_hands_medium-light_skin_tone":"Awesome! Very well done",
    "clapping_hands_medium_skin_tone":"Awesome! Very well done",
    "confused_face":"I am really confused",
    "confounded_face":"I am really confused",
    "crying_face":"sad and disappointed",
    "disappointed_face":"sad and disappointed",
    "downcast_face_with_sweat":"sad and disappointed",
    "drooling_face":"sad and disappointed",
    "enraged_face":"angry and disappointed",
    "expressionless_face":"sad and disappointed",
    "face_blowing_a_kiss":"Just loved it",
    "angry_face":"Really angry and disappointed",
    "angry_face_with_horns":"Really angry and disappointed",
    "anguished_face":"Really angry and disappointed",
    "anxious_face_with_sweat":"Really angry and disappointed",
    "astonished_face":"astonishing",
    "beaming_face_with_smiling_eyes":"Really happy and loved it",
    "cat_face":"Ok",
    "clown_face":"Ok",
    "cold_face":"sad and disappointed",
    "confounded_face":"confused and disappointed",
    "confused_face":"sad and disappointed",
    "cow_face":"Ok",
    "cowboy_hat_face":"Cool",
    "crying_face":"sad and disappointed",
    "disappointed_face":"sad and disappointed",
    "disguised_face":"sad and disappointed",
    "dog_face":"Happy and Satisfied",
    "dotted_line_face":"sad and disappointed",
    "downcast_face_with_sweat":"sad and disappointed",
    "dragon_face":"OK",
    "drooling_face":"sad and disappointed",
    "enraged_face":"sad and disappointed",
    "expressionless_face":"sad and disappointed",
    "face_blowing_a_kiss":"Just loved it",
    "face_exhaling":"sad and disappointed",
    "face_holding_back_tears":"sad and disappointed",
    "face_in_clouds":"Ok",
    "face_in_clouds":"Ok",
    "face_savoring_food":"Ok",
    "face_screaming_in_fear":"sad and disappointed",
    "face_vomiting":"sad and disappointed",
    "face_with_crossed-out_eyes":"sad and disappointed",
    "face_with_diagonal_mouth":"sad and disappointed",
    "face_with_hand_over_mouth":"sad and disappointed",
    "face_with_head-bandage":"sad and disappointed",
    "face_with_medical_mask":"sad and disappointed",
    "face_with_monocle":"sad and disappointed",
    "face_with_open_eyes_and_hand_over_mouth":"sad and disappointed",
    "face_with_open_mouth":"Surprised! Wow",
    "face_with_peeking_eye":"sad and disappointed",
    "face_with_raised_eyebrow":"Confused!",
    "face_with_rolling_eyes":"Confused!",
    "face_with_spiral_eyes":"sad and disappointed",
    "face_with_steam_from_nose":"sad and disappointed",
    "face_with_symbols_on_mouth":"sad and disappointed",
    "face_with_tears_of_joy":"Just loved it",
    "face_with_thermometer":"sad and disappointed",
    "face_with_tongue":"Laughing",
    "face_without_mouth":"sad and disappointed",
    "first_quarter_moon_face":"OK",
    "flushed_face":"sad and disappointed",
    "frowning_face":"sad and disappointed",
    "frowning_face":"sad and disappointed",
    "frowning_face_with_open_mouth":"sad and disappointed",
    "full_moon_face":"Ok",
    "grimacing_face":"sad and disappointed",
    "grinning_face":"Just loved it",
    "grinning_face_with_big_eyes":"Just loved it",
    "grinning_face_with_smiling_eyes":"Just loved it",
    "grinning_face_with_sweat":"Just loved it",
    "grinning_squinting_face":"Just loved it",
    "hot_face":"anxious",
    "hushed_face":"Wow!",
    "kissing_face":"Just loved it",
    "kissing_face_with_closed_eyes":"Just loved it",
    "kissing_face_with_smiling_eyes":"Just loved it",
    "loudly_crying_face":"sad and disappointed",
    "man_facepalming":"Ok",
    "man_facepalming_dark_skin_tone":"Ok",
    "man_facepalming_dark_skin_tone":"Ok",
    "man_facepalming_light_skin_tone":"Ok",
    "man_facepalming_light_skin_tone":"Ok",
    "man_facepalming_medium-dark_skin_tone":"Ok",
    "man_facepalming_medium-dark_skin_tone":"Ok",
    "man_facepalming_medium-light_skin_tone":"Ok",
    "man_facepalming_medium-light_skin_tone":"Ok",
    "man_facepalming_medium_skin_tone":"Ok",
    "man_facepalming_medium_skin_tone":"Ok",
    "melting_face":"sad and disappointed",
    "money-mouth_face":"Just loved it",
    "monkey_face":"Ok",
    "mouse_face":"Ok",
    "nauseated_face":"sad and disappointed",
    "nerd_face":"Ok",
    "neutral_face":"Ok",
    "new_moon_face":"Ok",
    "partying_face":"Cool",
    "pig_face":"ok",
    "pleading_face":"Ok",
    "rabbit_face":"Ok",
    "relieved_face":"Relieved",
    "sad_but_relieved_face":"Sad but relieved",
    "saluting_face":"You are awesome",
    "shaking_face":"Ok",
    "shushing_face":"Ok",
    "sleeping_face":"sad and disappointed",
    "sleepy_face":"sad and disappointed",
    "slightly_frowning_face":"sad and disappointed",
    "slightly_smiling_face":"Just loved it",
    "smiling_face_with_halo":"Just loved it",
    "smiling_face_with_heart-eyes":"Just loved it",
    "smiling_face_with_hearts":"Just loved it",
    "smiling_face_with_horns":"Just loved it",
    "smiling_face_with_open_hands":"Just loved it",
    "smiling_face_with_smiling_eyes":"Just loved it",
    "smiling_face_with_sunglasses":"Just loved it",
    "smiling_face_with_tear":"Sad and Confused",
    "smirking_face":"Ok",
    "sneezing_face":"sad and disappointed",
    "squinting_face_with_tongue":"Just loved it",
    "sun_with_face":"Just loved it",
    "thinking_face":"I am worried",
    "tiger_face":"Just loved it",
    "tired_face":"sad and disappointed",
    "unamused_face":"sad and disappointed",
    "upside-down_face":"sad and disappointed",
    "weary_face":"sad and disappointed",
    "wind_face":"Ok",
    "wind_face":"Ok",
    "winking_face":"Cool",
    "winking_face_with_tongue":"Cool",
    "woman_facepalming":"Ok",
    "woman_facepalming":"Ok",
    "woman_facepalming_dark_skin_tone":"Ok",
    "woman_facepalming_dark_skin_tone":"Ok",
    "woman_facepalming_light_skin_tone":"Ok",
    "woman_facepalming_light_skin_tone":"Ok",
    "woman_facepalming_medium-dark_skin_tone":"Ok",
    "woman_facepalming_medium-dark_skin_tone":"Ok",
    "woman_facepalming_medium-light_skin_tone":"Ok",
    "woman_facepalming_medium-light_skin_tone":"Ok",
    "woman_facepalming_medium_skin_tone":"Ok",
    "woman_facepalming_medium_skin_tone":"Ok",
    "woozy_face":"sad and disappointed",
    "worried_face":"sad and disappointed",
    "yawning_face":"sad and disappointed",
    "zany_face":"Being crazy and Happy!! Yay!",
    "zipper-mouth_face":"sad and disappointed",
    "hand_with_index_finger_and_thumb_crossed":"Just loved it",
    "hand_with_index_finger_and_thumb_crossed_dark_skin_tone":"Just loved it",
    "hand_with_index_finger_and_thumb_crossed_light_skin_tone":"Just loved it",
    "hand_with_index_finger_and_thumb_crossed_medium-dark_skin_tone":"Just loved it",
    "hand_with_index_finger_and_thumb_crossed_medium-light_skin_tone":"Just loved it",
    "hand_with_index_finger_and_thumb_crossed_medium_skin_tone":"Just loved it",
    "thumbs_down":"unhappy and disappointed",
    "thumbs_down_dark_skin_tone":"unhappy and disappointed",
    "thumbs_down_light_skin_tone":"unhappy and disappointed",
    "thumbs_down_medium-dark_skin_tone":"unhappy and disappointed",
    "thumbs_down_medium-light_skin_tone":"unhappy and disappointed",
    "thumbs_down_medium_skin_tone":"unhappy and disappointed",
    "thumbs_up":"Just loved it",
    "thumbs_up_dark_skin_tone":"Just loved it",
    "thumbs_up_light_skin_tone":"Just loved it",
    "thumbs_up_medium-dark_skin_tone":"Just loved it",
    "thumbs_up_medium-light_skin_tone":"Just loved it",
    "thumbs_up_medium_skin_tone":"Just loved it",
    "beaming_face_with_smiling_eyes":"Just loved it",
    "grinning_face":"Just loved it",
    "grinning_face_with_big_eyes":"Just loved it",
    "grinning_face_with_smiling_eyes":"Just loved it",
    "grinning_face_with_sweat":"Just loved it",
    "grinning_squinting_face":"Just loved it",
    "kissing_face":"Just loved it",
    "kissing_face_with_closed_eyes":"Just loved it",
    "kissing_face_with_smiling_eyes":"Just loved it",
    "smiling_cat_with_heart-eyes":"Just loved it",
    "beaming_face_with_smiling_eyes":"Just loved it",
    "cat_with_wry_smile":"Just loved it",
    "grinning_cat_with_smiling_eyes":"Just loved it",
    "grinning_face_with_smiling_eyes":"Just loved it",
    "kissing_face_with_smiling_eyes":"Just loved it",
    "slightly_smiling_face":"Just loved it",
    "smiling_cat_with_heart-eyes":"Just loved it",
    "smiling_face_with_halo":"Just loved it",
    "smiling_face_with_heart-eyes":"Just loved it",
    "smiling_face_with_hearts":"Just loved it",
    "smiling_face_with_horns":"Just loved it",
    "smiling_face_with_open_hands":"Just loved it",
    "smiling_face_with_sunglasses":"Just loved it",
    "smiling_face_with_tear":"Sad and Confused",
    "fire":"Just loved it",
    "angry_face":"angry and disappointed",
    "beaming_face_with_smiling_eyes":"Just loved it",
    "crying_face":"sad and disappointed",
    "disappointed_face":"sad and disappointed",
    "face_vomiting":"sad and disappointed",
    "face_with_hand_over_mouth":"Happy and Laughing",
    "face_with_open_eyes_and_hand_over_mouth":"Happy and Laughing",
    "pensive_face":"sad and disappointed",
    "hundred_points":"Nailed it!Loved it",
    "folded_hands_light_skin_tone": "Just loved it. You saved me",
    "victory_hand_light_skin_tone":"Thank you so much"
    # add more emoji mappings here
}

def replace_emoji_labels(text):
    for emoji_label, meaning in emoji_dict.items():
        emoji_character = emoji.emojize(emoji_label)
        text = text.replace(emoji_character, meaning)
    return text

def remove_repeat(sentence):
    
    # Remove repeated words
    words = re.findall(r'\b\w+\b', sentence.lower())
    cleaned_words = []
    for i, word in enumerate(words):
        if i == 0 or word != words[i-1]:
            cleaned_words.append(word)
    cleaned_sentence = ' '.join(cleaned_words)
    
    return cleaned_sentence

In [12]:
import re
from textblob import TextBlob
import textblob


def preprocess(text):
    if not isinstance(text, str):
        # Handle non-string values (e.g., NaN) by returning an empty string
        return ''
    
    # Convert to lowercase
    text = text.lower()
    
    # Convert emoji to text
    text = emoji.demojize(text)
    text = text.replace(":", "")
    text = replace_emoji_labels(text)
    text = remove_repeat(text)
    
    return text


In [13]:
df = pd.read_csv('Twitter_Data.csv') 
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [14]:
X = df['clean_text'].apply(preprocess)
y = df['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model = SentenceTransformer('sentence-transformers/nli-roberta-base-v2')

In [1]:
# Tokenize and encode sentences
X_train_embeddings = model.encode(X_train.tolist(), convert_to_tensor=True)
X_test_embeddings = model.encode(X_test.tolist(), convert_to_tensor=True)

NameError: name 'model' is not defined